# D-1 Group Project

## TODO:

1) read in data from `ASTR19_S22_group_project_data.txt`
2) define an oscillatory function in Python to describe intra- and inter-day variations in the tide
3) use scipy library to fit the oscillatory function to the data assuming the root mean squared experimental error on the height of the tide is 0.25 feet
4) graph both the model and the data on the same plot (label the data, provide sufficient labeling of the axes to provide clarity, save the figure to PDF)
5) subtract off the best fit function from the data and plot the residuals
6) ~more that I will add a bit later~

In [1]:
import scipy